<a href="https://colab.research.google.com/github/Mingjia000/Master_thesis/blob/main/ppo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time
from datetime import datetime

import torch
import torch.nn as nn
from torch.distributions import MultivariateNormal
from torch.distributions import Categorical

import numpy as np

In [ ]:
################################## set device ##################################

print("============================================================================================")


# set device to cpu or cuda
device = torch.device('cpu')

if(torch.cuda.is_available()): 
    device = torch.device('cuda:0') 
    torch.cuda.empty_cache()
    print("Device set to : " + str(torch.cuda.get_device_name(device)))
else:
    print("Device set to : cpu")
    
print("============================================================================================")

Device set to : cpu


In [ ]:
class RolloutBuffer:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []
    

    def clear(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]

In [ ]:
class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, action_std_init):
        super(ActorCritic, self).__init__()

        self.actor = nn.Sequential(
                            nn.Linear(state_dim, 64),
                            nn.ReLU(),
                            nn.Linear(64, 64),
                            nn.ReLU(),
                            nn.Linear(64, action_dim),
                            nn.Softmax(dim=-1)
                        )

        self.critic = nn.Sequential(
                            nn.Linear(state_dim, 64),
                            nn.ReLU(),
                            nn.Linear(64, 64),
                            nn.ReLU(),
                            nn.Linear(64, 1)
                    )
    def forward(self):
        raise NotImplementedError   

    def act(self, state):

        action_probs = self.actor(state.T)
        dist = Categorical(action_probs)

        action = dist.sample()
        action_logprob = dist.log_prob(action)
        
        return action.detach(), action_logprob.detach()      
   
    def evaluate(self, state, action):

        print('evaluate',state.T)

        action_probs = self.actor(state.T)
        
        dist = Categorical(action_probs)

        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        state_values = self.critic(state.T)
        
        return action_logprobs, state_values, dist_entropy     

In [ ]:
class PPO:
    def __init__(self, state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, action_std_init=0.6):

        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.buffer = RolloutBuffer()

        self.policy = ActorCritic(state_dim, action_dim, action_std_init).to(device)
        self.optimizer = torch.optim.Adam([
                        {'params': self.policy.actor.parameters(), 'lr': lr_actor},
                        {'params': self.policy.critic.parameters(), 'lr': lr_critic}
                    ])

        self.policy_old = ActorCritic(state_dim, action_dim, action_std_init).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        self.MseLoss = nn.MSELoss()

    def select_action(self, state):

          with torch.no_grad():
              state = torch.FloatTensor(state).to(device)
              action, action_logprob = self.policy_old.act(state)
            
          self.buffer.states.append(state)
          self.buffer.actions.append(action)
          self.buffer.logprobs.append(action_logprob)

          return action.item()


    def update(self):

        # Monte Carlo estimate of returns
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(self.buffer.rewards), reversed(self.buffer.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
            
        # Normalizing the rewards
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-7)

        # convert list to tensor
        old_states = torch.squeeze(torch.stack(self.buffer.states, dim=0)).detach().to(device)
        old_actions = torch.squeeze(torch.stack(self.buffer.actions, dim=0)).detach().to(device)
        old_logprobs = torch.squeeze(torch.stack(self.buffer.logprobs, dim=0)).detach().to(device)

        
        # Optimize policy for K epochs
        for _ in range(self.K_epochs):

            # Evaluating old actions and values
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)

            # match state_values tensor dimensions with rewards tensor
            state_values = torch.squeeze(state_values)
            
            # Finding the ratio (pi_theta / pi_theta__old)
            ratios = torch.exp(logprobs - old_logprobs.detach())

            # Finding Surrogate Loss
            advantages = rewards - state_values.detach()   
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages

            # final loss of clipped objective PPO
            loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards) - 0.01*dist_entropy
            
            # take gradient step
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
            
        # Copy new weights into old policy
        self.policy_old.load_state_dict(self.policy.state_dict())

        # clear buffer
        self.buffer.clear()
    
    
    def save(self, checkpoint_path):
        torch.save(self.policy_old.state_dict(), checkpoint_path)
   

    def load(self, checkpoint_path):
        self.policy_old.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))
        self.policy.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))
        

In [ ]:
################################### Training ###################################


####### initialize environment hyperparameters ######

env_name = "short path"
has_continuous_action_space = False

max_ep_len = 20                    # max timesteps in one episode 400
max_training_timesteps = int(1e5)   # break training loop if timeteps > max_training_timesteps

print_freq = max_ep_len * 4     # print avg reward in the interval (in num timesteps)
log_freq = max_ep_len * 2       # log avg reward in the interval (in num timesteps)
save_model_freq = int(2e4)      # save model frequency (in num timesteps)

action_std = None

################ PPO hyperparameters ################

update_timestep = max_ep_len * 4      # update policy every n timesteps
K_epochs = 40               # update policy for K epochs
eps_clip = 0.2              # clip parameter for PPO
gamma = 0.99                # discount factor

lr_actor = 0.0003       # learning rate for actor network
lr_critic = 0.001       # learning rate for critic network

random_seed = 0         # set random seed if required (0 = no random seed)

#####################################################


print("training environment name : " + env_name)

# state space dimension
state_dim = 1

# action space dimension
action_dim = 1

###################### logging ######################

#### log files for multiple runs are NOT overwritten

log_dir = "PPO_logs"
if not os.path.exists(log_dir):
      os.makedirs(log_dir)

log_dir = log_dir + '/' + env_name + '/'
if not os.path.exists(log_dir):
      os.makedirs(log_dir)


#### get number of log files in log directory
run_num = 0
current_num_files = next(os.walk(log_dir))[2]
run_num = len(current_num_files)


#### create new log file for each run 
log_f_name = log_dir + '/PPO_' + env_name + "_log_" + str(run_num) + ".csv"

print("current logging run number for " + env_name + " : ", run_num)
print("logging at : " + log_f_name)

#####################################################

################### checkpointing ###################

run_num_pretrained = 0      #### change this to prevent overwriting weights in same env_name folder

directory = "PPO_preTrained"
if not os.path.exists(directory):
      os.makedirs(directory)

directory = directory + '/' + env_name + '/'
if not os.path.exists(directory):
      os.makedirs(directory)


checkpoint_path = directory + "PPO_{}_{}_{}.pth".format(env_name, random_seed, run_num_pretrained)
print("save checkpoint path : " + checkpoint_path)


training environment name : short path
current logging run number for short path :  7
logging at : PPO_logs/short path//PPO_short path_log_7.csv
save checkpoint path : PPO_preTrained/short path/PPO_short path_0_0.pth


In [ ]:
def state_determin(action):
  new_state=action

  return new_state



In [ ]:
def reward_determine(old_state,state):
  time_matrix=np.array([[0,1,2,0,0,0],
                      [2,0,0,1,2,0],
                      [2,0,0,2,2,0],
                      [0,2,2,0,0,1],
                      [0,2,2,0,0,2],
                      [0,0,0,2,2,0]])
  if time_matrix[old_state,state]==0:
    penalty=100000
    
  reward=time_matrix[old_state,state]+penalty
  return reward


In [ ]:
def travel_time_determine(old_state,state):
    time_matrix = np.array([[0,1,2,0,0,0],
                      [2,0,0,1,2,0],
                      [2,0,0,2,2,0],
                      [0,2,2,0,0,1],
                      [0,2,2,0,0,2],
                      [0,0,0,2,2,0]])
    travel_time = time_matrix[old_state,state]
    return travel_time

In [ ]:
def done_determine(state):
  destination=6
  if state==destination:
    done=0
  else:
    done=1
  return done

In [ ]:
################# training procedure ################

# initialize a PPO agent
ppo_agent = PPO(state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, action_std)


# track total training time
start_time = datetime.now().replace(microsecond=0)
print("Started training at (GMT) : ", start_time)

print("============================================================================================")


# logging file
log_f = open(log_f_name,"w+")
log_f.write('episode,timestep,reward\n')


# printing and logging variables
print_running_reward = 0
print_running_episodes = 0

log_running_reward = 0
log_running_episodes = 0

time_step = 0
i_episode = 0


# training loop
while time_step <= max_training_timesteps:
    
    state = np.array([0]) #起点  terminal 0
    current_ep_reward = 0
    activation=1

    for t in range(1, max_ep_len+1):
        # exmaine the activation of states
        if  activation==1:
          # select action with policy
          action = ppo_agent.select_action(state)
          previous_state=state
          state = state_determin(action)
          reward = reward_determine(previous_state,state)#It is a floating data type value.
          done=done_determine(state)          #if 到达终点 done=1
          route_time=travel_time_determine(previous_state,state)
          arrival_t = t + route_time
               
          # saving reward and is_terminals
          ppo_agent.buffer.rewards.append(reward)
          ppo_agent.buffer.is_terminals.append(done)
          
          time_step +=1
          current_ep_reward += reward

          # update PPO agent
          if time_step % update_timestep == 0:
              print('update',state)
              ppo_agent.update()
              
          # log in logging file
          if time_step % log_freq == 0:

              # log average reward till last episode
              log_avg_reward = log_running_reward / log_running_episodes
              log_avg_reward = np.round(log_avg_reward, 4)

              log_f.write('{},{},{}\n'.format(i_episode, time_step, log_avg_reward))
              log_f.flush()

              log_running_reward = 0
              log_running_episodes = 0

          # printing average reward
          if time_step % print_freq == 0:

              # print average reward till last episode
              print_avg_reward = print_running_reward / print_running_episodes
              print_avg_reward = round(print_avg_reward, 2)

              print("Episode : {} \t\t Timestep : {} \t\t Average Reward : {}".format(i_episode, time_step, print_avg_reward))

              print_running_reward = 0
              print_running_episodes = 0
            
          # save model weights
          if time_step % save_model_freq == 0:
              print("--------------------------------------------------------------------------------------------")
              print("saving model at : " + checkpoint_path)
              ppo_agent.save(checkpoint_path)
              print("model saved")
              print("Elapsed Time  : ", datetime.now().replace(microsecond=0) - start_time)
              print("--------------------------------------------------------------------------------------------")
            
          # break; if the episode is over
          if done:
              break          

        # update activation 
        if t >= arrival_t:
          activation = 1
        else:
          activation = 0
        


    print_running_reward += current_ep_reward
    print_running_episodes += 1

    log_running_reward += current_ep_reward
    log_running_episodes += 1

    i_episode += 1


log_f.close()




# print total training time
print("============================================================================================")
end_time = datetime.now().replace(microsecond=0)
print("Started training at (GMT) : ", start_time)
print("Finished training at (GMT) : ", end_time)
print("Total training time  : ", end_time - start_time)
print("============================================================================================")




Started training at (GMT) :  2022-09-19 09:49:06
update 0
evaluate tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])


RuntimeError: ignored